In [ ]:
from ngsolve import *

from ngsolve.meshes import *
#from ngsolve.solvers import *
from ngsolve.webgui import Draw
from netgen.geom2d import unit_square

import numpy as np
#import matplotlib.pyplot as plt

#from su2_yangmills import *
from lgt import *
import time

In [ ]:
from pathlib import Path
from ngsolve.fem import CompilePythonModule

lgtcpp_path = "../../src/gaugelinkfe/lgt.cpp" 

if True:
    print("compile it")
    m = CompilePythonModule(Path(lgtcpp_path), init_function_name="lgt_module")
else:
    print("load it")
    #import lgt_module as m

In [ ]:
ne=5
#mesh = MakeStructured2DMesh(quads=True, nx=ne, ny=ne, periodic_x=True, periodic_y=True)
mesh = MakeStructured2DMesh(quads=True, nx=ne, ny=ne, periodic_x=False, periodic_y=False)
#mesh = MakeStr>uctured2DMesh(quads=False, nx=ne, ny=ne, periodic_x=False, periodic_y=False)
#mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))
Draw(mesh)
meshvars = m.MeshVars(mesh)

In [ ]:
#check_facet_vorb(meshvars)
#draw_tbone_weights(meshvars)
#draw_bone_weights(meshvars)

In [ ]:
order=0
#fesC = L2(mesh, order=order)**3 # for su2 charge
#fesJ = HDiv(mesh, order=order, discontinuous=True)**3 # for su2 charge current

# discrete connection one form sitting on borders
#fesA = FacetFESpace(mesh, order=0)**3

# parallel transport maps between neighbouring volumes, belonging to an oriented interface
#TODO: orientation?
fesU = FacetFESpace(mesh, order=0)**4

# quaternion space!
fesG = L2(mesh, order=0)**4

# space of wilson loops centered around bones (= corners (2D) or edges (3D))
# for visualization only!
fesW = H1(mesh, order=order)**4

# space of action integrand

fesS = H1(mesh, order=order)

In [ ]:
# list the basis matrices
# the coefficient basis is 1/i sigma_k
# or in other words isomorphic to the space of quaternions 
for a in range(0,4):
    Ua = np.zeros(4)
    Ua[a] = 1.
    #U = m.SU2(0., 0., 0., 1.)
    U = m.SU2(*Ua)
    print(f"generator nr. {a}:\n")
    print(U)
    print("\n")


In [ ]:
qtheta=0.
qtheta=0.5*pi
theta=0.
phi=0.

qU = get_rotq(qtheta, axis(theta, phi))
print(f"qU: \n {qU}")
U = m.SU2(*qU)
print(f"U:\n {U}")

print(f"U.conj(): \n {U.Conj()}")
print(f"U*U.Conj(): \n {U*U.Conj()}")

rqU = [U.GetCoeff(a) for a in range(0,4)]
print(f"reconstructed qU: \n {rqU}:")

In [ ]:
qtheta=0.5*pi
theta=0.
phi=0.

qU = get_rotq(qtheta, axis(theta, phi))
U1 = m.SU2(*qU)
#U2 = m.SU2(1./sqrt(2),0., 1./sqrt(2), 0.)
print(qU)
print(U1)

qtheta=0.5*pi
theta=0.5*pi
phi=0.

qU = get_rotq(qtheta, axis(theta, phi))
U2 = m.SU2(*qU)
#U2 = m.SU2(1./sqrt(2),0., 1./sqrt(2), 0.)
print(qU)
print(U2)


print(U1*U2)
print(U2*U1)

# TODO: implement - operator?
#print(U1*U2 - U2*U1)

print( (U1*U2) * (U1*U2).Conj() )

In [ ]:
# check conjugation, exp and log
n=100
eps = 0.000001

qtheta_max = pi
#phi_max = 0.5*pi
#qtheta_min = -qtheta_max
qtheta_min = -pi
qthetas = qtheta_min + (qtheta_max - qtheta_min)*np.random.rand(n)

thetas = 0.*pi + 1.*pi*np.random.rand(n)
phis = 0.*pi + 2.*pi*np.random.rand(n)

for i in range(n):
    qU = np.array(get_rotq(qthetas[i], axis(thetas[i], phis[i])))
    U = m.SU2(*qU)
    rqU = np.array([U.GetCoeff(a) for a in range(0,4)])
    
    if np.linalg.norm(qU - rqU) > eps:
        print("faulty implementation, qU and rqU do not match up")
        print(np.linalg.norm(qU - rqU))
        
    Uc = U.Conj()
    UUc = (U*Uc)
    qUUc = np.array([UUc.GetCoeff(a) for a in range(0,4)])
    
    if np.linalg.norm(qUUc - np.array([1., 0., 0., 0.])) > eps:
        print("faulty implementation, UUc and Id do not match up")
        print(np.linalg.norm(qUUc - np.array([1., 0., 0., 0.])))

In [ ]:
W = m.SU2(1., 0., 0., 10.)

print(W)
print(W.Conj())
print(W* (W.Conj()) )

m.CalcSW(W)

In [ ]:
gfU = GridFunction(fesU)

#gfU.components[0].vec[:] = 1

#phi  = 0.1*pi
#gfU.components[0].vec[:] = cos(phi)
#gfU.components[3].vec[:] = sin(phi)

#phi_max = 0
qtheta_max = 0.3*pi
#phi_max = 0.5*pi
#qtheta_min = -qtheta_max
qtheta_min = -0.3*pi
qthetas = qtheta_min + (qtheta_max - qtheta_min)*np.random.rand(mesh.nedge)

thetas = np.zeros(mesh.nedge)
phis = np.zeros(mesh.nedge)

for enum in range(mesh.nedge):
    # TODO: check python function
    set_gfU_link(gfU, enum, 0., 0., 0.)
    #set_gfU_link(gfU, enum, qthetas[enum], thetas[enum], phis[enum])
    U = m.SU2(*get_rotq(qthetas[enum], axis(thetas[enum], phis[enum])) )
    m.SetLink(gfU, enum, U, True)
    


    

# TODO: create cpp functions
# TODO: check python version of functions
gfW = GridFunction(fesW)
calc_gfW(gfU, gfW, mesh)
Draw(gfW[0], mesh)

In [ ]:
meshvars.RefreshBoneWeights()

In [ ]:
meshvars.GetBoneWeight(0)
meshvars.GettBoneWeight(0)

In [ ]:
gfS = GridFunction(fesS)
m.CalcgfS(meshvars, gfU, gfS)

gfW = GridFunction(fesW)
gfS_py = GridFunction(fesS)
calc_gfW(gfU, gfW, mesh)
# caution: python functions is wilson action without weights!
calc_wilson_action(gfW, gfS_py)

Draw(gfS, mesh)
#Draw(gfS_py, mesh)

In [ ]:
for e_id in range(mesh.nedge):
    #e_id = 47
    U = m.GetLink(gfU, e_id, True)
    e = mesh.edges[e_id]
    qU_py = get_qlink(gfU, e)
    U_py = m.SU2(*qU_py)

    print(U)
    print(U_py)
    print("\n")

#print(U_py.GetMatrix())

In [ ]:
#v_id = 35
#e_id = 47

v_id = 0
e_id = 0


e = mesh.edges[e_id]
#v = mesh.vertices[v_id]
print(e.vertices)
v = mesh.vertices[v_id]
print(v.edges)

In [ ]:
#v_id = 35
#v_id = 50

v_id = 8
v = mesh.vertices[v_id]


# TODO: check consistency of orientations used in py and cpp code
E_ors_py = get_oriented_vedges_tan(v, mesh)

E_ors_calc = meshvars.CalcVertexOrientedEnums(v_id)

E_ors = meshvars.GetOrientedEdgesOfVertex(v_id)

print(E_ors)
print(E_ors_py)


draw_orientededges(E_ors_py, mesh)
draw_orientededges(E_ors_calc, mesh)
draw_orientededges(E_ors, mesh)

In [ ]:
#v_id = 35
#e_id = 65

v_id = 50
e_id = 96
print(m.ArrangeOrientedEdgesOfVertex(E_ors, v_id, e_id, False))
print(m.ArrangeOrientedEdgesOfVertex(E_ors, v_id, e_id, True))

In [ ]:
v_id = 35
e_id = 65

v_id = 50
e_id = 96

v_id = 4
e_id = 4




#W_B = m.GetWilsonLoop(meshvars, gfU, v_id, e_id, True)
#W_B = m.GetWilsonLoop(meshvars, gfU, v_id, e_id, False)
# TODO: there is no guarantee that this ...
W_B = m.GetWilsonLoop(meshvars, gfU, v_id, -1, True)




v = mesh.vertices[v_id]
e = mesh.edges[e_id]
#qWv = get_wilson_loop(gfU, v, mesh, start_edge=e)
#qWv = get_wilson_loop(gfU, v, mesh, last_edge=e)
# ... and this should coincide ...
qWv = get_qwilson_loop(gfU, v, mesh, start_edge=None)

W_B_py = m.SU2(*qWv)




print(W_B)
print(W_B_py)

In [ ]:
for v_id in range(mesh.nv):
    #W_B = m.GetWilsonLoop(gfU, v_id, -1, True)
    
    # this is a bit problematic, as you need to fix the start edge in advance
    # which is not possible in the python code ...
    #W_B = m.GetWilsonLoop(meshvars, gfU, v_id, e_id, True)
    W_B = m.GetWilsonLoop(meshvars, gfU, v_id, -1, True)
    qW_B = [W_B.GetCoeff(a) for a in range(4)]

    v = mesh.vertices[v_id]
    #qW_py = get_qwilson_loop(gfU, v, mesh, start_edge=e)
    qW_py = get_qwilson_loop(gfU, v, mesh, start_edge=None)
    W_B_py = m.SU2(*qW_py)

    #print(W_B)
    #print(W_B_py)
    #print("\n")
    
    if np.linalg.norm(qW_B - qW_py) > eps:
        print("faulty implementation, qW_B and qW_py do not match up")
        print(np.linalg.norm(qW_B - qW_py))
    

In [ ]:
gfUold = GridFunction(fesU)
gfUold.vec.data[:] = gfU.vec.data[:]
gfUnew = GridFunction(fesU)

gfj_flux = calc_upwind_colorflux(CF( (0,0,0) ), CF( (1,0) ), gfU, mesh, glob_or=True)

dt = 0.1
#m.EOMUpdate(meshvars, gfU, gfUold, gfUnew, dt)
m.EOMUpdate(meshvars, gfU, gfUold, gfUnew, gfj_flux, dt)

In [ ]:
feshc = HCurl(mesh, order=0)
mhc = BilinearForm(feshc)
uhc,vhc = feshc.TnT()
mhc += InnerProduct(uhc,vhc)*dx
mhc.Assemble()
mhcmat = mhc.mat

fesh1 = H1(mesh, order=1)
mh1 = BilinearForm(fesh1)
uh1,vh1 = fesh1.TnT()
mh1 += InnerProduct(uh1,vh1)*dx
mh1.Assemble()
mh1mat = mh1.mat

M = mhcmat
Mt = mh1mat

v_weights = [Mt[_v_id, _v_id] for _v_id in range(mesh.nv)]
e_weights = [M[_e_id, _e_id] for _e_id in range(mesh.nedge)]

mesh_oriented_vedges = []
for v in mesh.vertices:
    #print(v)
    #print(type(v))
    mesh_oriented_vedges.append(get_oriented_vedges_tan(v, mesh))


In [ ]:
gfUnew_py = GridFunction(fesU)
py_timestep(gfU, gfUnew_py, gfUold, mesh, v_weights, e_weights, dt, vedges_list=mesh_oriented_vedges)

In [ ]:
gfS = GridFunction(fesS)
m.CalcgfS(meshvars, gfU,gfS)

gfS_py = GridFunction(fesS)
calc_gfW(gfU, gfW, mesh)
calc_wilson_action(gfW,gfS_py)

Draw(gfS, mesh)
Draw(gfS_py, mesh)
Draw(gfS-gfS_py, mesh)


In [ ]:
# apply gauge transformation
gfg = GridFunction(fesG)


el_num = 8


#_single_el_gauge(gfg, el_num, qtheta=0.25*pi, theta=0., phi=0.)
single_el_gauge(gfg, el_num, qtheta=pi, theta=0., phi=0.)

Draw(gfg.components[0], mesh)

gfU.components[0].vec.data[:] = 1.
gfU.components[1].vec.data[:] = 0.
gfU.components[2].vec.data[:] = 0.
gfU.components[3].vec.data[:] = 0.

fesHd = HDiv(mesh, order=0)

#print(sum(gfU.components[0].vec) - len(gfU.components[0].vec))
gfUviz = GridFunction(fesHd)
gfUviz1 = GridFunction(fesHd)

#gfUviz.vec.data[:] = gfU.components[0].vec[:]
#gfUviz1.vec.data[:] = 1.
gfUviz1.vec.data[:] = 0.
#Draw(gfUviz, mesh)

gfUviz.vec.data[:] = gfU.components[0].vec[:]
#gfUviz.vec.data[:] = 1.
##gfUviz.vec.data[:] = 0.




#Draw(gfUviz - gfUviz1, mesh)


print("before gauge")
Draw(gfUviz, mesh)


gauge_gfU(gfU, gfg, fesU)



gfUviz.vec.data[:] = gfU.components[0].vec[:]
#gfUviz.vec.data[:] = 0.

print("aftergauge")
Draw(gfUviz, mesh)

#print(sum(gfU.components[0].vec) - len(gfU.components[0].vec))
#print(gfU.components[0].vec)